# Calculate PI
Using Monte Calro, calculating approximate pi value.

Written on icsharp kernel

In [49]:
using System;
using System.Math;
using System.Random;
using System;
using System.Diagnostics;
using System.Threading;
using System.Threading.Thread;
Console.WriteLine(Environment.ProcessorCount);

16

In [ ]:
static class Log{
    static int count = 0;
    public static Stopwatch sw = new Stopwatch();
    static Log(){
        sw.Start();
    }
    public static void Write(object obj){
        count++;
        if(count%10==0)
            Console.Clear();
        Console.Write("[{0}] {1}",sw.Elapsed, obj);
    }
}

class Worker{
    public long TrueCount;
    public long FalseCount;
    public int ChunchCount = 100000;
    Random rand;
    public Worker(){
        var seed = (int)Log.sw.ElapsedTicks;
//         Log.Write(seed);
        rand = new Random(seed);
    }
    public void Run(){
        for(int i=0; i<ChunchCount; i++){
            var x = rand.NextDouble();
            var y = rand.NextDouble();
            var d = x*x+y*y;
            if(d < 1){
                TrueCount++;
            }else{
                FalseCount++;
            }
        }
    }
}

var workers = new List<Worker>();
for(int i=0; i<Environment.ProcessorCount; i++){
    Thread.Sleep(7);
    workers.Add(new Worker());
}

object totalLock = new object();
int total = 0;
Parallel.For(0, Environment.ProcessorCount, (i)=>{
    while(total < 100000 * 30){
        workers[i].Run();
        lock(totalLock){
            total++;
            //Console.Write(total);
        }
    }
});

long tc, fc;
foreach(var item in workers){
    tc += item.TrueCount;
    fc += item.FalseCount;
}
var size = (double)tc / (tc + fc);
var pi = size * 4;
Log.Write($"done. aprox. pi {pi}");
Log.Write($"differnce. {Math.Abs(Math.PI - pi)}");